In [2]:
import numpy as np
import pandas as pd

In [3]:
df_all = pd.read_csv("csv/processed_data_concat.csv")

In [36]:
df_all['Metadata_Plate'] = df_all['Metadata_Plate'].apply(str)
df_all['Metadata_Well'] = df_all['Metadata_Well'].str.upper()
df_all["SAMPLE_KEY"]= df_all[['Metadata_Plate', 'Metadata_Well']].agg('-'.join, axis=1)
df_all_small = pd.concat([df_all[['SAMPLE_KEY', 'Metadata_broad_sample']],df_all.iloc[:,17:-1]], axis=1)

In [18]:
df_smiles = pd.read_csv("csv/chemical_annotations.csv")
df_smiles = df_smiles.rename(columns={"BROAD_ID":"Metadata_broad_sample"})

In [57]:
from rdkit import Chem
from rdkit.Chem.inchi import MolToInchiKey

def smiles_to_mol(smiles):
    """ Convert a SMILES string to INCHIKEY
    """
    if not smiles:
        return ""
    try:
        m = Chem.MolFromSmiles(smiles)
    except:
        return ""
    inchikey = MolToInchiKey(m)
    return inchikey

In [58]:
df_merge = pd.merge(df_smiles[["Metadata_broad_sample", "CPD_SMILES"]], 
         df_all_small[df_all_small["Metadata_broad_sample"]!="DMSO"], 
         how = "right",
         on="Metadata_broad_sample")
df_merge.insert(0,'INCHIKEY','')
df_merge['INCHIKEY'] = df_merge['CPD_SMILES'].apply(smiles_to_mol)

In [62]:
df_merge = df_merge.drop(columns="Metadata_broad_sample")

In [63]:
df_merge

,INCHIKEY,CPD_SMILES,SAMPLE_KEY,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,PRTMXUMLGYLWPX-UHFFFAOYSA-N,Cc1ccccc1C(=O)NNC(=S)NC(=O)c1ccccc1,25372-A03,5438.0,352.0,220.0,1.341633,0.852598,1.0,0.548659,...,3.865420,1.406476,1.530319,1.495116,1.711729,1.930069,1.870354,2.698786,2.544510,2.573773
1,JQIGIYGSOWUONU-UHFFFAOYSA-N,Clc1ccc(Cn2nc(-c3ccccc3)c(cc2=O)N2CCCCC2)cc1,25372-A04,4347.0,350.0,247.0,1.400722,0.860154,1.0,0.529412,...,3.593072,1.205263,1.282520,1.231165,1.668451,1.774585,1.712926,2.602744,2.441468,2.532226
2,JBZPLGMTBVEFIZ-UHFFFAOYSA-N,COc1ccc(cc1)-c1c[n+]2CC(=C3CCCCn1c23)c1ccc(Br)cc1,25372-A05,4150.0,428.0,261.0,1.367345,0.863943,1.0,0.553578,...,4.311241,1.363952,1.356302,1.329784,2.600722,2.386389,2.381538,2.554461,2.387792,2.455155
3,UFNAEUOXECFKRF-UHFFFAOYSA-N,CC(C)NC(=O)ON=C(C)c1sc(nc1C)-c1ccccc1,25372-A06,1681.0,314.0,261.0,1.239217,0.786957,1.0,0.557732,...,3.457100,1.790952,1.786865,1.766933,1.729822,1.871437,1.840501,2.169722,2.261197,2.277600
4,PXIJCSZNPGOKCC-UHFFFAOYSA-O,CC\C(Nc1ccccc1)=C/c1oc2cc3ccccc3cc2[n+]1CC,25372-A07,3631.0,364.0,254.0,1.318691,0.845600,1.0,0.543647,...,3.456242,1.558367,1.628596,1.596774,1.642393,1.751831,1.706001,2.514233,2.434023,2.468646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126793,DAHNEQAICFEUKA-RZIGYZOXSA-N,C[C@@H](CO)N1C[C@@H](C)[C@@H](CN(C)Cc2ccc(cc2)...,24740-P20,3348.0,347.5,253.0,1.335052,0.844260,1.0,0.530161,...,2.644437,1.553295,1.595929,1.595249,1.468006,1.574379,1.541481,2.288113,2.361514,2.413726
126794,DSHXGOUIOHRIOZ-DNVFCKCGSA-N,COc1ccc(CN(C)C[C@H]2OCc3cn(CCCC(=O)N(C[C@H]2C)...,24740-P21,3866.5,389.0,285.5,1.340830,0.838347,1.0,0.548940,...,2.651812,1.530398,1.451356,1.481157,1.734706,1.745322,1.746123,2.255154,2.228869,2.270517
126795,QVHCTLOUCAXGOY-UMEGOILYSA-N,C[C@H](CO)N1C[C@@H](C)[C@@H](CN(C)S(=O)(=O)c2c...,24740-P22,3471.5,399.0,257.0,1.390091,0.864196,1.0,0.530796,...,2.692879,1.644374,1.620638,1.657840,1.400277,1.449004,1.414502,2.360146,2.342338,2.390572
126796,DSHXGOUIOHRIOZ-YDHSSHFGSA-N,COc1ccc(CN(C)C[C@@H]2OCc3cn(CCCC(=O)N(C[C@@H]2...,24740-P23,3696.0,355.0,253.0,1.298613,0.818742,1.0,0.556117,...,2.608098,1.409582,1.466746,1.503614,1.530055,1.739891,1.662961,2.218604,2.336516,2.393916


In [ ]:
df_merge.to_csv("temp/cellpainting-index.csv", index=False)